In [1]:
from wilcoxon.spiderman import *
import re
import pandas as pd
from tqdm import tqdm

In [28]:
main = website("https://docs.microsoft.com/en-us/powerquery-m/power-query-m-function-reference")
submains = ["https://docs.microsoft.com/en-us/powerquery-m/" + x['href'] for x in main.html(attrs={"data-linktype":"relative-path"})[1:]]
subsubmains = []
for submain in tqdm(submains):
    submain = website(submain)
    subsubmains.extend(["https://docs.microsoft.com/en-us/powerquery-m/" + x['href'] for x in submain.html(attrs={"data-linktype":"relative-path"})])


100%|██████████| 24/24 [00:11<00:00,  2.04it/s]


In [86]:
def getData(link):
    site = website(link)
    siteInfo = {"Link":link}
    try:
        subject = ''.join(list(site.html.find("h2",text="Syntax").find_next("pre").strings)).replace("\n","")
        siteInfo['Subject'] = subject
    except:
        siteInfo['Subject'] = site.html.find("h1").text
    try:
        about = ''.join(list(site.html.find("h2",text="About").find_next("p").strings)).replace("\n","")
        siteInfo['Explanation'] = about
    except: None
    try:
        explanation = ''.join(list(site.html.find("h2",text=re.compile("Example")).find_next("p").strings)).replace("\n","")
        example = ''.join(list(site.html.find("h2",text=re.compile("Example")).find_next("pre").strings)).replace("\n","")
        siteInfo['Example'] = explanation + "\n" + example
    except: None
    return siteInfo


In [87]:
data = []
for subsubmain in tqdm(subsubmains):
    data.append(getData(subsubmain))

100%|██████████| 753/753 [05:30<00:00,  2.28it/s]


In [90]:
df = pd.DataFrame(data,columns = ['Software', 'Subject', 'Type', 'Explanation', 'Return', 'Example', 'Remarks', 'Package', 'Level', 'Link'])

In [94]:
df['Software'] = "M"
df['Type'] = "function"

In [96]:
df.to_clipboard()